In [ ]:
# BACKEND: app.py
cat > src/backend/app.py <<'PYAPP'
from flask import Flask, request, jsonify
from flask_cors import CORS
import os, io
from PIL import Image

# Import inference functions (placeholder)
from inference import predict_image, predict_audio, answer_question

app = Flask(__name__)
CORS(app)

@app.route('/predict_image', methods=['POST'])
def predict_image_route():
    if 'file' not in request.files:
        return jsonify({'error':'no file part'}), 400
    f = request.files['file']
    try:
        img = Image.open(io.BytesIO(f.read())).convert('RGB')
        tmp = os.path.join('..','..','data','images','_tmp_backend.jpg')
        os.makedirs(os.path.dirname(tmp), exist_ok=True)
        img.save(tmp)
        label, conf = predict_image(tmp)
        return jsonify({'label': label, 'confidence': conf})
    except Exception as e:
        return jsonify({'error': str(e)}), 500

@app.route('/predict_audio', methods=['POST'])
def predict_audio_route():
    if 'file' not in request.files:
        return jsonify({'error':'no file part'}), 400
    f = request.files['file']
    tmp = os.path.join('..','..','data','audio','_tmp_backend.wav')
    os.makedirs(os.path.dirname(tmp), exist_ok=True)
    with open(tmp, 'wb') as out:
        out.write(f.read())
    label, conf = predict_audio(tmp)
    return jsonify({'label': label, 'confidence': conf})

@app.route('/qa', methods=['POST'])
def qa_route():
    data = request.get_json() or {}
    q = data.get('question', '')
    answer = answer_question(q)
    return jsonify({'answer': answer})

if __name__ == '__main__':
    port = int(os.environ.get('PORT', 8000))
    app.run(host='0.0.0.0', port=port, debug=True)
PYAPP